In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [15]:
bc_history = pd.read_csv('../input/bitstampUSD_1-min_data_2012-01-01_to_2019-03-13.csv')
bc_history_nonan = bc_history.dropna(thresh=2)

In [17]:
bc_history.head()

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
0,1325317920,4.39,4.39,4.39,4.39,0.455581,2.0,4.39
1,1325317980,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1325318040,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1325318100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1325318160,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
bc_history_nonan.head()

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
0,1325317920,4.39,4.39,4.39,4.39,0.455581,2.000000,4.390000
478,1325346600,4.39,4.39,4.39,4.39,48.000000,210.720000,4.390000
547,1325350740,4.50,4.57,4.50,4.57,37.862297,171.380338,4.526411
548,1325350800,4.58,4.58,4.58,4.58,9.000000,41.220000,4.580000
1224,1325391360,4.58,4.58,4.58,4.58,1.502000,6.879160,4.580000


## Data Preprocessing

There are two possibilities for missing data in our dataset (as explained in the dataset description):
1. When at a given timestamp there is no trade or activity the field will be filled with NaN value
2. When there are some technical error (such as the API is down) then the timestamp is missing (the dataset are recorded on each minutes

### Selecting data

In here, we're going to select dataset with minimal missing timestamp jump

In [13]:
bc_history_nonan['Delta_Timestamp'] = bc_history_nonan['Timestamp'] - bc_history_nonan['Timestamp'].shift(1)
bc_history_nonan['Delta_Timestamp']

3773153    180.0
3773154     60.0
3773155     60.0
3773156     60.0
3773157     60.0
3773158     60.0
3773159     60.0
3773160     60.0
3773163    180.0
3773164     60.0
3773165     60.0
3773170    300.0
3773172    120.0
3773173     60.0
3773174     60.0
3773175     60.0
3773176     60.0
3773177     60.0
3773178     60.0
3773179     60.0
3773180     60.0
3773181     60.0
3773182     60.0
3773183     60.0
3773184     60.0
3773185     60.0
3773186     60.0
3773187     60.0
3773188     60.0
3773189     60.0
           ...  
3778779     60.0
3778780     60.0
3778781     60.0
3778782     60.0
3778783     60.0
3778786    180.0
3778787     60.0
3778788     60.0
3778789     60.0
3778790     60.0
3778791     60.0
3778792     60.0
3778793     60.0
3778794     60.0
3778795     60.0
3778796     60.0
3778797     60.0
3778799    120.0
3778800     60.0
3778801     60.0
3778802     60.0
3778804    120.0
3778806    120.0
3778807     60.0
3778809    120.0
3778810     60.0
3778811     60.0
3778814    180

In [14]:
bc_history_nonan['Delta_Timestamp'].tail(5000)

3773153    180.0
3773154     60.0
3773155     60.0
3773156     60.0
3773157     60.0
3773158     60.0
3773159     60.0
3773160     60.0
3773163    180.0
3773164     60.0
3773165     60.0
3773170    300.0
3773172    120.0
3773173     60.0
3773174     60.0
3773175     60.0
3773176     60.0
3773177     60.0
3773178     60.0
3773179     60.0
3773180     60.0
3773181     60.0
3773182     60.0
3773183     60.0
3773184     60.0
3773185     60.0
3773186     60.0
3773187     60.0
3773188     60.0
3773189     60.0
           ...  
3778779     60.0
3778780     60.0
3778781     60.0
3778782     60.0
3778783     60.0
3778786    180.0
3778787     60.0
3778788     60.0
3778789     60.0
3778790     60.0
3778791     60.0
3778792     60.0
3778793     60.0
3778794     60.0
3778795     60.0
3778796     60.0
3778797     60.0
3778799    120.0
3778800     60.0
3778801     60.0
3778802     60.0
3778804    120.0
3778806    120.0
3778807     60.0
3778809    120.0
3778810     60.0
3778811     60.0
3778814    180

From the data above, we know 5000 lastest data only missing some empty values (this including timestamp jump and NaN values), so we will use it for processing.

In [20]:
bc_history = bc_history.tail(5000)
bc_history.head()

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
3773817,1552135260,3902.62,3903.11,3902.62,3903.11,0.482676,1883.936114,3903.105223
3773818,1552135320,3903.11,3903.11,3903.11,3903.11,4.727675,18452.636311,3903.110000
3773819,1552135380,3903.11,3903.37,3903.11,3903.37,1.150000,4488.615500,3903.143913
3773820,1552135440,3903.11,3903.46,3901.65,3901.65,5.993798,23395.765313,3903.329266
3773821,1552135500,3901.65,3903.25,3899.54,3902.59,8.798698,34331.851306,3901.924206


### Missing values replacement

Replace NaN field with previous data for continuous timeseries data

In [ ]:
bc_history['Open'].fillna(method='ffill', inplace=True)
bc_history['High'].fillna(method='ffill', inplace=True)
bc_history['Low'].fillna(method='ffill', inplace=True)
bc_history['Close'].fillna(method='ffill', inplace=True)

Replace NaN field with zeros for single event data

In [ ]:
bc_history['Volume_(BTC)'].fillna(value=0, inplace=True)
bc_history['Volume_(Currency)'].fillna(value=0, inplace=True)
bc_history['Weighted_Price'].fillna(value=0, inplace=True)